### Install necessary libraries

In [ ]:
# to install opencv
pip install opencv-python

#to install cmake
pip install cmake

#to install dlib
pip install dlib

#### Download `shape_predictor_68_face_landmarks.dat` 

In [ ]:
import requests
import bz2

# Download the `shape_predictor_68_face_landmarks.dat` compressed file

url = "http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2"
response = requests.get(url)

# Save the compressed file
with open("shape_predictor_68_face_landmarks.dat.bz2", "wb") as f:
    f.write(response.content)

# Extract the .dat file from the .bz2 archive
with bz2.open("shape_predictor_68_face_landmarks.dat.bz2", "rb") as f_in:
    with open("shape_predictor_68_face_landmarks.dat", "wb") as f_out:
        f_out.write(f_in.read())


#### Run below code to see liveness detection

In [5]:
import cv2
import dlib
import numpy as np
from scipy.spatial import distance as dist

# Initialize dlib's face detector and facial landmark predictor
face_detector = dlib.get_frontal_face_detector()
landmark_predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

# Eye aspect ratio (EAR) threshold and consecutive frames threshold for blink detection
EAR_THRESHOLD = 0.2
CONSEC_FRAMES = 3

def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

def get_landmarks(gray):
    """
    Detects facial landmarks and returns coordinates for key points.
    """
    faces = face_detector(gray)
    if len(faces) == 0:
        return None
    landmarks = []
    face = faces[0]
    shape = landmark_predictor(gray, face)
    for i in range(68):  # 68 landmark points
        x, y = shape.part(i).x, shape.part(i).y
        landmarks.append((x, y))
    return np.array(landmarks, dtype=np.int32)

def calculate_displacement(landmarks_prev, landmarks_curr):
    """
    Calculates the displacement of facial landmarks between two frames.
    """
    if landmarks_prev is None or landmarks_curr is None:
        return 0
    displacements = np.linalg.norm(landmarks_curr - landmarks_prev, axis=1)
    return np.sum(displacements)  # Total displacement

def detect_liveness(sequence_length=20):
    cap = cv2.VideoCapture(0)
    blink_count = 0
    consecutive_frames = 0
    prev_landmarks = None
    movement_history = []
    total_blink_count = 0
    WINDOW_SIZE = (400, 400)  # Smaller window size
    CENTER_RADIUS = 150       # Radius of the circular area for face alignment

    while total_blink_count <= 2:
        ret, frame = cap.read()
        if not ret:
            break
            
        # Resize frame for smaller window
        frame = cv2.resize(frame, WINDOW_SIZE)
        
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        landmarks = get_landmarks(gray)
        
        # Create circular mask
        mask = np.zeros_like(frame)
        center = (WINDOW_SIZE[0] // 2, WINDOW_SIZE[1] // 2)
        cv2.circle(mask, center, CENTER_RADIUS, (255, 255, 255), -1)
        frame = cv2.bitwise_and(frame, mask)  # Apply mask to frame
        
        
        # Eye landmarks (right eye: 36-41, left eye: 42-47)
        if landmarks is not None:
            left_eye = landmarks[36:42]
            right_eye = landmarks[42:48]
            
            # Calculate EAR for both eyes
            left_EAR = eye_aspect_ratio(left_eye)
            right_EAR = eye_aspect_ratio(right_eye)
            ear = (left_EAR + right_EAR) / 2.0

            # Check for blink
            if ear < EAR_THRESHOLD:
                consecutive_frames += 1
            else:
                if consecutive_frames >= CONSEC_FRAMES:
                    blink_count += 1
                    total_blink_count += 1
                consecutive_frames = 0

            # Head movement detection
            if prev_landmarks is not None:
                displacement = calculate_displacement(prev_landmarks, landmarks)
                movement_history.append(displacement)
                
                # Draw landmarks and display displacement
                for (x, y) in landmarks:
                    cv2.circle(frame, (x, y), 2, (0, 255, 0), -1)
                    #cv2.putText(frame, f"Displacement: {displacement:.2f}", (10, 30),
                            #cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

            # Update previous landmarks
            prev_landmarks = landmarks

        # Display live blink count and movement
        cv2.putText(frame, f"PLace your face in the circle & blink eye ", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.55, (255, 150, 100), 2)
        cv2.putText(frame, f"Blink Count: {blink_count}", (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

        cv2.imshow("Liveness Detection", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    # Calculate total head movement
    total_movement = sum(movement_history)
    print(f"Total Head Movement (cumulative displacement): {total_movement}")
    print(f"Total Blink Count: {total_blink_count}")

    # Classification based on thresholds
    if total_movement > 1000 and total_blink_count == 3:
        print("Classification: Live")
    else:
        print("Classification: Spoof")

# Run the combined liveness detection
detect_liveness()


Total Head Movement (cumulative displacement): 102257.24897455503
Total Blink Count: 3
Classification: Live
